In [5]:
import json
import time
import pandas as pd

In [8]:
TOP_100_PATH = '../data/popular_100_memes.csv'
TOP_100 = pd.read_csv(TOP_100_PATH, encoding = "ISO-8859-1")

TOP_100.head()

/Users/ericsun/Documents/Cornell 2020 Fall/ai-meme-generator/Meme Generator/data/popular_100_memes.csv


,ID,Name,Alternate Names
0,112126428,Distracted Boyfriend,"distracted bf, guy checking out another girl, ..."
1,438680,Batman Slapping Robin,NaN
2,87743020,Two Buttons,"2 red buttons, choice button, which button, da..."
3,181913649,Drake Hotline Bling,"drakeposting, drakepost, drake hotline approve..."
4,61579,One Does Not Simply,"one does not simply walk into morder, lord of ..."
